In [1]:
!pip install --upgrade --quiet  langchain-community
!pip install --upgrade --quiet langchainhub langgraph

In [2]:
import sqlite3

import requests
from langchain_community.utilities.sql_database import SQLDatabase
from sqlalchemy import create_engine
from sqlalchemy.pool import StaticPool


def get_engine_for_chinook_db():
    """Pull sql file, populate in-memory database, and create engine."""
    url = "https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sql"
    response = requests.get(url)
    sql_script = response.text

    connection = sqlite3.connect(":memory:", check_same_thread=False)
    connection.executescript(sql_script)
    return create_engine(
        "sqlite://",
        creator=lambda: connection,
        poolclass=StaticPool,
        connect_args={"check_same_thread": False},
    )


engine = get_engine_for_chinook_db()

db = SQLDatabase(engine)

In [3]:
!pip install -qU langchain-google-genai

In [14]:
import getpass
import os

os.environ["GOOGLE_API_KEY"] = getpass.getpass()
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [7]:

os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
os.environ["LANGSMITH_TRACING"] = "true"

In [15]:
from langchain import hub

prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")

assert len(prompt_template.messages) == 1
print(prompt_template.input_variables)

['dialect', 'top_k']


In [16]:
system_message = prompt_template.format(dialect="SQLite", top_k=5)

In [17]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(
    llm, toolkit.get_tools(), state_modifier=system_message
)

Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'default' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring


In [18]:
example_query = "Which country's customers spent the most?"

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Which country's customers spent the most?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (af65bc7d-6129-4b3e-a28d-8d3b56bf704b)
 Call ID: af65bc7d-6129-4b3e-a28d-8d3b56bf704b
  Args:
    tool_input:
================================= Tool Message =================================
Name: sql_db_list_tables

Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (3b2ef2e6-77c5-4885-a61b-2d8ea8c1825c)
 Call ID: 3b2ef2e6-77c5-4885-a61b-2d8ea8c1825c
  Args:
    table_names: Customer, Invoice
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"

================================== Ai Message ==================================
Tool Calls:
  sql_db_query (acf7795a-b8cd-4c1b-962f-0b03c04afe8d)
 Call ID: acf7795a-b8cd-4c1b-962f-0b03c04afe8d
  Args:
    query: SELECT T2.Country, SUM(T1.Total) AS TotalSpent FROM Invoice AS T1 INNER JOIN Customer AS T2 ON T1.CustomerId = T2.CustomerId GROUP BY T2.Country ORDER BY TotalSpent DESC LIMIT 5
================================= Tool Message =================================
Name: sql_db_query

[('USA', 523.0600000000003), ('Canada', 303.9599999999999), ('France', 195.09999999999994), ('Brazil', 190.09999999999997), ('Germany', 156.48)]
================================== Ai Message ==================================

The country with the highest customer spending is USA.


In [19]:
example_query = "Who are the top 3 best selling artists?"

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Who are the top 3 best selling artists?


================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (3078043e-5667-4333-bb5a-fac508ac6d5f)
 Call ID: 3078043e-5667-4333-bb5a-fac508ac6d5f
  Args:
    tool_input:
================================= Tool Message =================================
Name: sql_db_list_tables

Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (4926f040-a632-4e03-a1e7-9e442f818118)
 Call ID: 4926f040-a632-4e03-a1e7-9e442f818118
  Args:
    table_names: Artist, Album
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")
)

/*
3 rows from Album

In [20]:
example_query = "Who are the top 3 best selling artists?"

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Who are the top 3 best selling artists?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (2b891b40-d1d6-482e-a4ee-b281f835136d)
 Call ID: 2b891b40-d1d6-482e-a4ee-b281f835136d
  Args:
    tool_input:
================================= Tool Message =================================
Name: sql_db_list_tables

Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (85f5fd4d-7f0c-44e5-9afa-6f9fa9ec5b03)
 Call ID: 85f5fd4d-7f0c-44e5-9afa-6f9fa9ec5b03
  Args:
    table_names: Artist
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


================================= Tool Message =================================
Name: sql_db_query_checker

```sql
SELECT T2.Name, SUM(T1.Quantity) AS TotalSales FROM InvoiceLine AS T1 INNER JOIN Track AS T2 ON T1.TrackId = T2.TrackId GROUP BY T2.Name ORDER BY TotalSales DESC LIMIT 3
```
================================== Ai Message ==================================
Tool Calls:
  sql_db_query (cd675689-262f-4e34-81e1-61421c6830d2)
 Call ID: cd675689-262f-4e34-81e1-61421c6830d2
  Args:
    query: SELECT T2.Name, SUM(T1.Quantity) AS TotalSales FROM InvoiceLine AS T1 INNER JOIN Track AS T2 ON T1.TrackId = T2.TrackId GROUP BY T2.Name ORDER BY TotalSales DESC LIMIT 3
================================= Tool Message =================================
Name: sql_db_query

[('The Trooper', 5), ('Untitled', 4), ('The Number Of The Beast', 4)]
================================== Ai Message ==================================

The top 3 best selling artists are The Trooper, Untitled, and The Number O